In [43]:
from xml.dom.minidom import (parse, Element)

In [44]:
corpus_filename = 'Tselina'
#corpus_filename = 'pedagogika'
#corpus_filename = 'sirija'

file_ext = '.xml'
doc = parse(corpus_filename + file_ext)
sentences = doc.getElementsByTagName('S') # все элементы с тегом S (все предложения корпуса)

In [45]:
f'Количество предложений в корпусе = {len(sentences)}'

'Количество предложений в корпусе = 189'

In [46]:
d = {} # словарь всей грамматики граммема - слова

class Sentence:
    def __init__(self, sent: Element):
        self.id = sent.getAttribute('ID')
        self.words = []
        
        self.parseWords(sent.getElementsByTagName('W'))
        # получаем все элементы xml с тегом W (слова)
        
    def parseWords(self, rawWords):
        
        self.wordMap = {} # для последующего связывания слов
        # ключ - доминатор(айди доминатора) этого слова
        # значение - список слов, для которых доминатор равен ключу
        
        for rawWord in rawWords:
            
            word = Word(rawWord, self.id) # переносим в класс
            
            self.words.append(word) # добавляем в общий список слов
            
            if word.dom == '_root':
                self.rootWord = word # корневое слово
                
            # добавление в мапу
            if word.dom in self.wordMap.keys():
                self.wordMap[word.dom].append(word)
            else:
                self.wordMap[word.dom] = [word]

        # цикл по всем словам, чтобы связать их с помощью мапы
        for word in self.words:
            if word.id in self.wordMap.keys():
                for d in self.wordMap[word.id]:
                    word.addWord(d)
    def printGram(self):
        # печать грамматики предложения = печать грамматики одного слова(которое печатает все свои связанные слова, то есть все слова в предложении)
        self.rootWord.printGram(True) 
        
        
class Word:
    def __init__(self, w: Element, sentId):
        self.dom = w.getAttribute('DOM')
        self.feat = w.getAttribute('FEAT')
        self.id = w.getAttribute('ID')
        self.lemma = w.getAttribute('LEMMA')
        self.link = w.getAttribute('LINK')
        
        v = " ".join(t.nodeValue for t in w.childNodes if t.nodeType == t.TEXT_NODE).lower().strip()
        # v - просто слово
        
        if len(v) != 0 and v[-1] == '.':
            v = v[:-1]
        if v != '':
            # добавление слова в словарь
            if self.feat in d.keys():
                d[self.feat].add(v)
            else:
                d[self.feat] = {v}
        
        
        self.sentId = sentId
        
        self.connectedWords = [] #связанные с этим словом слова
        
    def addWord(self, w):
        # добавление слова в связанные с этим словом слова
        self.connectedWords.append(w)
        
    def printGram(self, checkProj):
        if checkProj:
            t = checkProjectivity(self, {}, 0)
            if not t:
                # закомментировать return, чтобы такие предложения не пропускались
                #print(f'\nNOT PROJECTIVE, {self.sentId}\n')
                return
        print('F{' + f'[{self.feat}' + ']}->', end = '') # F(t) ->
        
        if self.connectedWords == []:
            print(f'[{self.feat}]\n', end = '') # когда узел - терминальный
            
        else:
            t = False # для расположения граммем слова в нужном месте
            for w in self.connectedWords:
                
                # проход по всем словам в связанных словах
                if not t and (int(w.id) > int(self.id)): 
                    # когда этого не происходило ранее и когда айди текущего связанного слова превысило айди самого слова
                    print(f'[{self.feat}]', end = '')
                    t = True
                    
                print(';D{[' + f'{w.feat}' + '], ' + f'{w.link}' + '};', end = '') # D(t, s)
            if not t:
                print(f'[{self.feat}]', end = '')
            for w in self.connectedWords:
                print('\nD{[' + f'{w.feat}], {w.link}' + '}->F{[' + f'{w.feat}' + ']}') #D(t, s) -> F(t)
                #print() # раскомментировать и закомментировать предыдущую, если не хотим печатать D(t, s) -> F(t)
                w.printGram(False)
                
def checkProjectivity(rootWord, seenWordsIds, floor):
    if floor not in seenWordsIds.keys():
        # создаем список для этого яруса
        seenWordsIds[floor] = []
    if any(map(lambda x: int(x) > int(rootWord.id), seenWordsIds[floor])):
        # если в истории этого яруса есть id больший, чем id текущего узла
        return False
    seenWordsIds[floor].append(rootWord.id) # добавляем в историю id текущего узла
    if rootWord.connectedWords == []: # если нет поддеревьев
        return True
    t = True
    for w in rootWord.connectedWords:
        # проверяем все поддеревья
        t = t and checkProjectivity(w, seenWordsIds, floor + 1)
        if not t:
            # если случай найден, то возвращаем сразу же
            return t
    return t   
        

In [47]:
def parseSentences():
    for sent in sentences:
        # пробегаемся по всем предложениям корпуса
        parseSentence(sent)

def parseSentence(sent: Element):
    sentence = Sentence(sent)
    sentence.printGram()

In [48]:
import sys
filename = corpus_filename + '_' + 'gram.out'

In [49]:
orig_stdout = sys.stdout
f = open(filename, 'w', encoding = 'utf-16')
sys.stdout = f
# перенаправление stdout в файл, чтобы все печаталось в файл

parseSentences()

# перенаправление обратно
sys.stdout = orig_stdout
f.close()

In [50]:
filename_sorted = corpus_filename + '_' + 'gram_sorted.out'

In [51]:
# сортировка и удаление дубликатов в файле
def printSortedNoDuplicatesFile(fname, fname_sorted):
    lines_seen = set()
    with open(fname, 'r', encoding = 'utf-16') as r:
        with open(fname_sorted, 'w', encoding = 'utf-16') as f:
            for line_orig in sorted(r):
                if line_orig not in lines_seen:
                    # чистим от пробелов
                    line = str(line_orig.strip())

                    if len(line) != 0 and line[-1] == ';':
                        # удаляю лишние ;
                        line = line[:-1].strip()
                        
                    line = line.replace(';;', ';')
                    line = line.replace('->;', '->')
                    
                    line += '\n'  
                    if line == '\n':
                        # empty line
                        continue
                    
                    print(line, end = '', file = f)
                    
                    lines_seen.add(line_orig)

In [52]:
printSortedNoDuplicatesFile(filename, filename_sorted)

In [53]:
# печать словаря
def printDictionary(d, fname):
    with open(fname, 'w', encoding='utf-16') as f:
        for key, value in d.items():
            s = ' | '.join(value)
            print(f'[{key}] = {s}', file = f)

In [54]:
printDictionary(d, f'{corpus_filename}_dict.out')

In [55]:
d_gram_map = {}
f_gram_map = {}

In [56]:
def addToMap(mapName, key, value):
    # добавление в словарь по ключу key value (само значение - список)
    if key in mapName.keys():
        mapName[key].append(value)
    else:
        mapName[key] = [value]

def mapGrammar(filename):
    # запись всей грамматики в словарь
    # d_gram_map - словарь для D нетерминалов
    # f_gram_map - словарь для F нетерминалов
    
    with open(filename, 'r', encoding = 'utf-16') as r:
        for line in r:
            s = line.split('->')
            p = s[0].strip()
            w = s[1].strip().replace('\n', '')
            if (p[0] == 'D'):
                addToMap(d_gram_map, p, w)
            else:
                addToMap(f_gram_map, p, w)

In [57]:
def printGrammarMap(mapName, filename_min):
    # печать грамматики
    with open(filename_min, 'w', encoding = 'utf-16') as f:
        for key, value in mapName.items():
            s = '|'.join(value)
            s = ';'.join(s.split(';'))
            print(key, '->', s, file = f, sep = '')

In [58]:
def setFToD():
    # подстановка F нетерминалов в D
    for key, value in d_gram_map.items():
        d_gram_map[key] = f_gram_map[d_gram_map[key][0]]

In [59]:
def removeRecursion(d_gram_map):
    # подставить D в D где это возможно
    seen = []
    t = True
    while t: 
        t = False # закончится, если справа не останется D, или все правила просмотрены и обработаны(останутся только те, в которых есть рекурсии)
        for key, value in d_gram_map.items():
            if 'D{' not in str(value) and key not in seen: # если справа есть нетерминал D
                t = True
                seen.append(key)
                
                for k, v in d_gram_map.items():
                    l = []
                    if key in str(v):
                        for item in v:
                            for val in value:
                                l.append(item.replace(key, val)) # заменяю D на D из других правил
                    if l != []:
                        d_gram_map[k] = l
                        
    return d_gram_map
            

In [60]:
import copy

In [61]:
d_gram_map = {}
f_gram_map = {}


mapGrammar(f'{corpus_filename}_gram_sorted.out')
    
setFToD()

new_map = removeRecursion(copy.deepcopy(d_gram_map))
printGrammarMap(new_map, f'{corpus_filename}_gram_min.out')
printSortedNoDuplicatesFile(f'{corpus_filename}_gram_min.out', f'{corpus_filename}_gram_min_sorted.out')

In [62]:
class RTNNode:
    # класс для рекурсивной сети переходов
    # s - терминал или нетерминал
    # i - в какое состояние переходим по этому нетерминалу/терминалу(цифра после)
    # isEnd - последнее ли правило в альтернативе
    # connected - с каким RTNNode связано текущее
    def __init__(self, s, i, isEnd = False):
        self.s = s
        self.i = i
        self.isEnd = isEnd
        self.connected = None
        
    # связать с другим RTNNode
    def connect(self, node):
        self.connected = node
        
    # печать одного RTNNode
    def printNode(self):
        
        s = f'${self.s}' if self.s[0] == 'D' else self.s
        
        if self.isEnd:
            print(s, '*', end = '')
            print(';')
        else:
            print(s, self.i, end = '')
            print(';')
           
    # печать RTNNode если printItself True, иначе напечатать все связанные с этим RTNNode 
    def printNodes(self, printItself = False):
        if printItself:
            self.printNode()
        if not self.isEnd:
            print(f'{self.i}:')
        if self.connected != None:
            self.connected.printNodes(printItself = True)
        

def toRTN(new_map, fname):
    
    orig_stdout = sys.stdout
    f = open(fname, 'w', encoding = 'utf-16')
    sys.stdout = f
    
    for key in new_map.keys():
        l = new_map[key] # ['a;D1', 'D2;D1']
        new_l = [] # [['a', 'D1'], ['D2', 'D1']]
        for item in l:
            p = item.split(';')
            new_l.append(p)
            
        print(f'${key}')
        print('(')
        
        j = 0
        k = 0
        t = 0
        m = {}
        
        start_ones = []
        
        for items in new_l:
            
            last = None # RTNNode на предыдущей итерации
            
            if k == 0:
                k = len(items) - 1 # для самого первого ставим длину всего списка
            else:
                k += t # иначе добавляем t
                
            j = k # индекс по которому будем переходить
            t = 0
            
            for i, item in enumerate(reversed(items)):
                # цикл по всем терминалам и нетерминалам в альтернативе в обратном порядке, чтобы успешно связать индекс состояния 
                if i == 0:
                    r = RTNNode(item, 0, isEnd = True)
                    last = r 
                    continue
                
                r = RTNNode(item, j)
                
                j -= 1 # уменьшаем j
                t += 1 # инкрементируем t
            
                if last != None:
                    r.connect(last) # связываем текущий с предыдущей итерацией
                    
                last = r
                
            start_ones.append(last)
        
        # печать всего RTNNode
        print('0:')
        # печать 0 состояния
        for i in start_ones:
            i.printNode()
        # печать всех остальных
        for i in start_ones:
            i.printNodes()  
                  
        print(')')
        print()
        
    sys.stdout = orig_stdout
    f.close()
                        

In [63]:
toRTN(new_map, f'{corpus_filename}_rtn.out')